# Projeto: Análise de Pedidos de um E-commerce 🛒

## Descrição do Problema
Uma empresa de e-commerce quer analisar seus dados de pedidos para entender melhor seus clientes, produtos e tendências de vendas. Seu objetivo é processar um dataset de pedidos usando **Pandas**, organizar os dados em um **banco de dados SQL** com **SQLAlchemy**, e realizar consultas para extrair insights úteis.

## Objetivos do Projeto
1. **Carregar e limpar os dados** de pedidos do e-commerce usando Pandas.
2. **Criar tabelas SQL** com SQLAlchemy para armazenar os dados de forma estruturada.
3. **Fazer consultas SQL** utilizando **ORM do SQLAlchemy** para responder perguntas como:
   - Quais são os produtos mais vendidos?
   - Quais clientes gastam mais?
   - Como a distribuição de preços dos produtos influencia as vendas?
4. **Implementar classes Python** para estruturar melhor a manipulação dos pedidos.

## Etapas
1. **Exploração dos dados com Pandas**
2. **Transformação dos dados em tabelas SQL**
3. **Consultas SQL utilizando ORM**
4. **Uso de classes para manipular os pedidos**

## Dados Utilizados
Os dados contêm informações de pedidos, incluindo:
- `order_id`: ID do pedido
- `customer_id`: ID do cliente
- `product`: Nome do produto
- `category`: Categoria do produto
- `price`: Preço do produto
- `quantity`: Quantidade comprada
- `order_date`: Data da compra

Vamos começar! 🚀

In [14]:

import pandas as pd
from sqlalchemy import create_engine, Column, Integer, String, Float, Date, func
from sqlalchemy.orm import declarative_base, sessionmaker

# Configuração inicial
Base = declarative_base()


In [15]:

# Criando um dataset fictício de pedidos
data = {
    "order_id": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    "customer_id": [101, 102, 101, 103, 104, 105, 101, 102, 103, 106],
    "product": ["Laptop", "Mouse", "Teclado", "Monitor", "Mousepad", "Cadeira Gamer", "Headset", "Mouse", "Laptop", "Teclado"],
    "category": ["Eletrônicos", "Acessórios", "Acessórios", "Eletrônicos", "Acessórios", "Móveis", "Acessórios", "Acessórios", "Eletrônicos", "Acessórios"],
    "price": [4500, 150, 300, 1200, 50, 900, 250, 150, 4500, 300],
    "quantity": [1, 2, 1, 2, 3, 1, 2, 1, 1, 2],
    "order_date": pd.to_datetime(["2024-01-01", "2024-01-02", "2024-01-03", "2024-01-04", "2024-01-05", "2024-01-06", "2024-01-07", "2024-01-08", "2024-01-09", "2024-01-10"])
}

# Criando o DataFrame
df = pd.DataFrame(data)
df["order_date"] = df["order_date"].dt.date
df


,order_id,customer_id,product,category,price,quantity,order_date
0,1,101,Laptop,Eletrônicos,4500,1,2024-01-01
1,2,102,Mouse,Acessórios,150,2,2024-01-02
2,3,101,Teclado,Acessórios,300,1,2024-01-03
3,4,103,Monitor,Eletrônicos,1200,2,2024-01-04
4,5,104,Mousepad,Acessórios,50,3,2024-01-05
5,6,105,Cadeira Gamer,Móveis,900,1,2024-01-06
6,7,101,Headset,Acessórios,250,2,2024-01-07
7,8,102,Mouse,Acessórios,150,1,2024-01-08
8,9,103,Laptop,Eletrônicos,4500,1,2024-01-09
9,10,106,Teclado,Acessórios,300,2,2024-01-10


In [16]:

# Criando a conexão com o SQLite usando SQLAlchemy
engine = create_engine("sqlite:///ecommerce.db")
Session = sessionmaker(bind=engine)
session = Session()


In [17]:

# Definição das classes para mapear as tabelas

class Order(Base):
    __tablename__ = 'orders'

    order_id = Column(Integer, primary_key=True)
    customer_id = Column(Integer)
    product = Column(String)
    category = Column(String)
    price = Column(Float)
    quantity = Column(Integer)
    order_date = Column(Date)
    
    def __repr__(self):
        return f"<Order(order_id={self.order_id}, product='{self.product}', price={self.price})>"
    
# Criando a tabela no banco
Base.metadata.create_all(engine)


In [18]:

# Inserindo os dados do DataFrame no banco SQL
df.to_sql("orders", con=engine, if_exists="replace", index=False)

# Testando uma consulta para visualizar os dados inseridos usando ORM
session.query(Order).all()


[<Order(order_id=1, product='Laptop', price=4500)>,
 <Order(order_id=2, product='Mouse', price=150)>,
 <Order(order_id=3, product='Teclado', price=300)>,
 <Order(order_id=4, product='Monitor', price=1200)>,
 <Order(order_id=5, product='Mousepad', price=50)>,
 <Order(order_id=6, product='Cadeira Gamer', price=900)>,
 <Order(order_id=7, product='Headset', price=250)>,
 <Order(order_id=8, product='Mouse', price=150)>,
 <Order(order_id=9, product='Laptop', price=4500)>,
 <Order(order_id=10, product='Teclado', price=300)>]

In [19]:

# 1. Produtos mais vendidos usando ORM
most_sold = session.query(
    Order.product, func.sum(Order.quantity).label("total_sold")
).group_by(Order.product).order_by(func.sum(Order.quantity).desc()).all()

most_sold


[('Teclado', 3),
 ('Mousepad', 3),
 ('Mouse', 3),
 ('Monitor', 2),
 ('Laptop', 2),
 ('Headset', 2),
 ('Cadeira Gamer', 1)]

In [20]:

# 2. Clientes que mais gastaram
top_customers = session.query(
    Order.customer_id, func.sum(Order.price * Order.quantity).label("total_spent")
).group_by(Order.customer_id).order_by(func.sum(Order.price * Order.quantity).desc()).all()

top_customers


[(103, 6900), (101, 5300), (105, 900), (106, 600), (102, 450), (104, 150)]

In [21]:

# 3. Analisando a distribuição de preços dos produtos usando Pandas
df.describe()


,order_id,customer_id,price,quantity
count,10.00000,10.00000,10.00000,10.000000
mean,5.50000,102.80000,1230.00000,1.600000
std,3.02765,1.75119,1761.02874,0.699206
min,1.00000,101.00000,50.00000,1.000000
25%,3.25000,101.25000,175.00000,1.000000
50%,5.50000,102.50000,300.00000,1.500000
75%,7.75000,103.75000,1125.00000,2.000000
max,10.00000,106.00000,4500.00000,3.000000


In [22]:

# Criando uma classe para manipular pedidos usando ORM

class OrderManager:
    def __init__(self, session):
        self.session = session
    
    def get_orders_by_customer(self, customer_id):
        return self.session.query(Order).filter(Order.customer_id == customer_id).all()

    def get_orders_by_product(self, product):
        return self.session.query(Order).filter(Order.product == product).all()

# Testando a classe
order_manager = OrderManager(session)
order_manager.get_orders_by_customer(101)


[<Order(order_id=1, product='Laptop', price=4500)>,
 <Order(order_id=3, product='Teclado', price=300)>,
 <Order(order_id=7, product='Headset', price=250)>]